# Assignment 3
# Name - Pranav Gupta
# Roll No. - 2021101095

In [1]:
import numpy as np
import cv2
import os
import pandas as pd
import argparse
import imutils
import sys

### 1. Face detection and association-based tracking

### 1.1 Video Preparation
Command used - youtube-dlc -k "https://www.youtube.com/watch?v=bSMxl1V8FSg"

#### 1.2 Face Detection

In [3]:
sys.argv = ["your_script.py", "-c", './cascades_dir']
ap = argparse.ArgumentParser()
ap.add_argument("-c", "--cascades", type=str, default="cascades",
    help="path to input directory containing haar cascades")
args = vars(ap.parse_args())

cascades_path = args["cascades"]

In [4]:
detectorPaths = {
	"face": "haarcascade_frontalface_default.xml",
	"eyes": "haarcascade_eye.xml",
	"smile": "haarcascade_smile.xml",
}
detectors = {}
for (name, path) in detectorPaths.items():
	path = os.path.sep.join([args["cascades"], path])
	detectors[name] = cv2.CascadeClassifier(path)

In [5]:
frames = []

for img_path in sorted(os.listdir('./output_frames')):
	frame = cv2.imread('./output_frames/'+img_path)
	frame = imutils.resize(frame, width=500)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	faceRects = detectors["face"].detectMultiScale(
		gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30),
		flags=cv2.CASCADE_SCALE_IMAGE)
	
	for (fX, fY, fW, fH) in faceRects:
		faceROI = gray[fY:fY+ fH, fX:fX + fW]
		cv2.rectangle(frame, (fX, fY), (fX + fW, fY + fH),
			(0, 255, 0), 2)
		
	cv2.imshow("Frame", frame)
	frames.append(frame)
	key = cv2.waitKey(1) & 0xFF
	if key == ord("q"):
		break

cv2.destroyAllWindows()

It takes almost 40 seconds to process 720 Frames. That means it takes only 0.11 seconds on an average to process each frame in the video. 
The size of the input frame, the complexity of the scene (number of faces, lighting, occlusions), and the parameters used in the detectMultiScale function (such as scaleFactor, minNeighbors, and minSize) are important variables that could alter the amount of time it takes to process each frame. Processing speed and detection accuracy can be traded off by adjusting these parameters. Processing time can also be affected by the hardware that is utilised to run the algorithm.

#### 1.3 Face Detection Visualisation

In [10]:
def video_gen_func(frames): 
	video_name = 'new_video_faces.avi'
	height, width, layers = frames[0].shape
	fps = 1
	video = cv2.VideoWriter(video_name, 0, fps, (width, height)) 
	for frame in frames: 
		video.write(frame)
	cv2.destroyAllWindows()
	video.release()


video_gen_func(frames)

Link to the Video generated using OpenCV Detectors - [Link](https://iiitaphyd-my.sharepoint.com/:v:/g/personal/pranav_g_students_iiit_ac_in/Ec2M1zJngilOlbAmoFCh2VwBVHozf4xrII4w5SmFtgcNXA?nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJTdHJlYW1XZWJBcHAiLCJyZWZlcnJhbFZpZXciOiJTaGFyZURpYWxvZy1MaW5rIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXcifX0%3D&e=KkiVwm)

3 conclusions why face detector fails are as follows - 

1. **Rapidly moving objects in continuous frames** -  It is because as the objects move rapdily in continuous frames, detector leads to capturing of false positions in the image where gradients change rapidly, and as a result, face detector marks the presence of a face.

2. **Scene involving the Face is too short** - That is, at one instance, the camera points at the girl and the boy while at the next frame it points at the 3 boys. This leads to Face Detector Failing to respond to quick changes in surroundings.

3. **When Objects are moving** - When objects are moving or turning in continuous frames, even then, the detector is not able to capture the change in gradients which leads to loss of information of the presence/absence of faces, which again leads to failure of detector is recognising a face or recognising extra windows as faces.

#### 1.4 Association-based Tracking

In [8]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

tracks = {}
track_id_counter = 0
tracked_frame = []
for filename in sorted(os.listdir('./output_frames')):
    frame_path = os.path.join('./output_frames', filename)
    frame = cv2.imread(frame_path)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    current_tracks = {}
    
    for (x, y, w, h) in faces:
        face_box = (x, y, x+w, y+h)
        matched_track_id = None
        
        for track_id, track_data in tracks.items():
            last_face_box = track_data[-1]
            intersection_area = (min(face_box[2], last_face_box[2]) - max(face_box[0], last_face_box[0]))*(min(face_box[3], last_face_box[3]) - max(face_box[1], last_face_box[1]))
            if intersection_area > 0:
                union_area = (w * h) + ((last_face_box[2] - last_face_box[0]) * (last_face_box[3] - last_face_box[1])) - intersection_area
                iou_score = intersection_area / union_area
                if iou_score > 0.5:
                    matched_track_id = track_id
                    break
        
        if matched_track_id is None:
            track_id_counter += 1
            current_tracks[track_id_counter] = [face_box]
        else:
            current_tracks[matched_track_id] = tracks[matched_track_id] + [face_box]
            del tracks[matched_track_id]
    
    for track_id, face_boxes in current_tracks.items():
        for face_box in face_boxes:
            x, y, x2, y2 = face_box
            cv2.rectangle(frame, (x, y), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, str(track_id), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    tracks = current_tracks
    tracked_frame.append(frame)

In [9]:
# Calculate the number of unique tracks created
unique_tracks = set()
for track_id in tracks.keys():
    unique_tracks.add(track_id)

print("Number of unique tracks created in the first 30 seconds:", track_id)

Number of unique tracks created in the first 30 seconds: 505


In [10]:
output_video_filename = "association_output_video.mp4"
height, width, _ = tracked_frame[0].shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter(output_video_filename, fourcc, 25, (width, height))  

for frame in tracked_frame:
    out.write(frame)
out.release()
print(f"Video saved as {output_video_filename}")

Video saved as association_output_video.mp4


Link to the Video generated using Association Based Tracking - [Link](https://iiitaphyd-my.sharepoint.com/:v:/g/personal/pranav_g_students_iiit_ac_in/EdI32MeuCRpEpmtxutv5dVMB-LvouZ3QU_2qq6ohHjEP_w?nav=eyJyZWZlcnJhbEluZm8iOnsicmVmZXJyYWxBcHAiOiJTdHJlYW1XZWJBcHAiLCJyZWZlcnJhbFZpZXciOiJTaGFyZURpYWxvZy1MaW5rIiwicmVmZXJyYWxBcHBQbGF0Zm9ybSI6IldlYiIsInJlZmVycmFsTW9kZSI6InZpZXcifX0%3D&e=fn2cxF)

Each unique track id corresponds to a unique character in the Video however, as soon as the character moves or turns or the action changes, multiple Track IDs are assigned to the characters. Quality of the Face Tracking is good and is able to capture even the slightest variations in face movements and expressions but the drawback is in the video, multiple objects in the background are also assigned Track IDs. There are many timestamps of failure cases, and it is because the Detector falsely classifies objects in the surroundings as faces and hence performs a poor jobs in the assignment of multiple track IDs to "False Faces". 

### 2 YOLO Object Detection

In [13]:
import numpy as np
from ultralytics import YOLO
import os
from google.colab import drive
import torch
import torch.nn as nn
import torchsummary as summary
import cv2
import shutil
import time
drive.mount('/content/drive')

In [1]:
!pip3 install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.6/749.6 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

### 2.1 Data Preparation

In [14]:
images_train = []
images_val = []
for image in os.listdir('images/train'):
# for image in os.listdir('/content/drive/MyDrive/images/train'):
    images_train.append('images/train/'+image)
# for image in os.listdir('/content/drive/MyDrive/images/val'):
for image in os.listdir('images/val'):
    images_val.append('images/val/'+image)

### 2.2 Understanding YOLO Object Detector

YOLO and R-CNN are both object identification algorithms and address the problem in quite different ways:

Single-Shot Detectors: These are quick and effective. They use a single neural network to process the entire image at once. Because of this, they are perfect for real-time applications. They might, however, be less precise than some two-stage techniques.

Detectors of R-CNN (such as Faster R-CNN): These are slower but more accurate. They employ a two-step process: they categorise the areas after first identifying possible object regions (region suggestions). The computational cost of this may be high.

3 versions of YOLO and the differences between them are:

YOLOv1 (2015): It was the first detector in the YOLO Series. It was fast but not the most accurate. It divided the image into a grid and predicted bounding boxes and class probabilities for each grid cell. While efficient, it struggled with small objects and multiple objects in a single cell.

YOLOv3 (2018): It introduced new techniques like feature maps from different scales to improve detection of objects of various sizes. It was also faster and more accurate than its predecessor. It solved some of the issues which YOLOv1 faced at that time.

YOLOv8 (2023): YOLOv8 focuses on both speed and accuracy, achieving state-of-the-art performance. YOLOv8 introduces the C2f module specifically to improve small object detection. Also, YOLOv8 takes Wasserstein Distance Loss addressing the issue of class imbalances for potentially better performance.

In essence, YOLO has continuously strived to improve accuracy without sacrificing speed, making it a strong contender in the world of real-time object detection.

### 2.3 Hands on with Ultralytics (Building YOLO Object Detector)

In [15]:
class YOLOScratch(nn.Module):

    def __init__(self):
        super(YOLOScratch, self).__init__()

        # Load a model
        self.model_nano_pt = YOLO('yolov8n.pt')  # load an official model
        self.model_nano_scratch = YOLO('yolov8n.yaml')  # load an official model
        self.model_medium_pt = YOLO('yolov8m.pt')  # load an official model

    def analyse(self):
        print("YOLOv8 Nano Model Parameters: ", self.model_nano_scratch.info())
        print("YOLOv8 Medium Model Parameters: ", self.model_medium_pt.info())
        print("YOLOv8 Nano Model: \n\n\n", self.model_nano_scratch)
        print("YOLOv8 Medium Model: \n\n\n", self.model_medium_pt)


model = YOLOScratch()
model.analyse()

YOLOv8n summary: 225 layers, 3157200 parameters, 3157184 gradients, 8.9 GFLOPs
YOLOv8 Nano Model Parameters:  (225, 3157200, 3157184, 8.8575488)
YOLOv8m summary: 295 layers, 25902640 parameters, 0 gradients, 79.3 GFLOPs
YOLOv8 Medium Model Parameters:  (295, 25902640, 0, 79.3204224)
YOLOv8 Nano Model: 


 YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNo

##### Comparison of YOLOv8n and YOLOv8m Models

YOLOv8n Model has 3157200 parameters while YOLOv8m Model has 25902640 parameters.
YOLOv8n Model uses 60 Convolutional Layers while YOLOv8m Model also uses the same number (60) of Convolutional Layers.

### 2.4 Training YOLO Variants

In [ ]:
# the folders created with this code snippet were uploaded on Google Drive 

if (os.path.exists('./custom_images1')):
    shutil.rmtree('./custom_images1')

indices = np.random.choice(400, 100, replace=False)
indices_dict = {}
for index in indices:
    indices_dict[index] = True
j = 0
custom_images = []
custom_labels = []
for i, img in enumerate(sorted(os.listdir('./images/train'))):
    if (i in indices_dict):
        custom_images.append(img)
        l = img.split('.')
        custom_labels.append(l[0]+'.txt')
        j += 1
    if (j == 100):
        break
os.mkdir('./custom_images1')

for file_name in custom_images:
    src = os.path.join('./images/train', file_name)
    dest = os.path.join('./custom_images1', file_name)
    shutil.copy(src, dest)

for file_name in custom_labels:
    src = os.path.join('./labels/train', file_name)
    dest = os.path.join('./custom_images1', file_name)
    shutil.copy(src, dest)



if (os.path.exists('./custom_images2')):
    shutil.rmtree('./custom_images2')


indices = np.random.choice(400, 100, replace=False)
indices_dict = {}
for index in indices:
    indices_dict[index] = True
j = 0
custom_images = []
custom_labels = []
for i, img in enumerate(sorted(os.listdir('./images/train'))):
    if (i in indices_dict):
        custom_images.append(img)
        l = img.split('.')
        custom_labels.append(l[0]+'.txt')
        j += 1
    if (j == 100):
        break
os.mkdir('./custom_images2')

for file_name in custom_images:
    src = os.path.join('./images/train', file_name)
    dest = os.path.join('./custom_images2', file_name)
    shutil.copy(src, dest)

for file_name in custom_labels:
    src = os.path.join('./labels/train', file_name)
    dest = os.path.join('./custom_images2', file_name)
    shutil.copy(src, dest)

In [5]:
results = model.model_nano_scratch.train(data = '/content/drive/MyDrive/config.yaml', epochs=20, imgsz=640)

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

100%|██████████| 755k/755k [00:00<00:00, 15.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/labels/train.cache... 400 images, 1 backgrounds, 0 corrupt: 100%|██████████| 401/401 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      20/20      2.29G      2.176      2.789      2.763          1        640: 100%|██████████| 26/26 [00:13<00:00,  1.94it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.18it/s]

                   all         50         52      0.309      0.423      0.217     0.0768



20 epochs completed in 0.100 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.10it/s]


                   all         50         52      0.305      0.423      0.217     0.0769
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train


In [ ]:
results = model.model_nano_pt.train(data = '/content/drive/MyDrive/config.yaml', epochs=20, imgsz=640)

Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train32, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

train: Scanning /content/drive/MyDrive/labels/train.cache... 400 images, 1 backgrounds, 0 corrupt: 100%|██████████| 401/401 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train32/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train32
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      2.79G     0.8454      1.284      1.145          1        640: 100%|██████████| 26/26 [00:13<00:00,  1.93it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.69it/s]

                   all         50         52      0.772      0.712      0.771      0.584



20 epochs completed in 0.089 hours.
Optimizer stripped from runs/detect/train32/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train32/weights/best.pt, 6.2MB

Validating runs/detect/train32/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.64it/s]


                   all         50         52      0.772      0.712      0.771      0.585
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to runs/detect/train32


In [ ]:
results = model.model_medium_pt.train(data = '/content/drive/MyDrive/config.yaml', epochs=20, imgsz=640)

Ultralytics YOLOv8.1.43 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

100%|██████████| 755k/755k [00:00<00:00, 21.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

train: Scanning /content/drive/MyDrive/labels/train... 400 images, 1 backgrounds, 0 corrupt: 100%|██████████| 401/401 [03:01<00:00,  2.21it/s]


train: New cache created: /content/drive/MyDrive/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      20/20      7.68G     0.9466      1.378      1.273          1        640: 100%|██████████| 26/26 [00:15<00:00,  1.63it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.88it/s]

                   all         50         52      0.711      0.711       0.67      0.528



20 epochs completed in 0.113 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 52.0MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics YOLOv8.1.43 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.05it/s]


                   all         50         52      0.711      0.712       0.67      0.528
Speed: 0.2ms preprocess, 9.2ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train10


In [ ]:
results = model.model_nano_scratch.train(data = '/content/drive/MyDrive/custom_dataset_config.yaml', epochs=20, imgsz=640)

Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/custom_dataset_config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

100%|██████████| 755k/755k [00:00<00:00, 15.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/custom_images2... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:03<00:00, 26.50it/s]

train: New cache created: /content/drive/MyDrive/custom_images2.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      2.29G      2.815      4.265      3.589          4        640: 100%|██████████| 7/7 [00:01<00:00,  4.53it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.58it/s]


                   all         50         52    0.00273      0.769     0.0143     0.0044

20 epochs completed in 0.024 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.73it/s]


                   all         50         52    0.00247      0.673     0.0288      0.014
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 3.8ms postprocess per image
Results saved to runs/detect/train


In [ ]:
results = model.model_nano_pt.train(data = '/content/drive/MyDrive/custom_dataset_config.yaml', epochs=20, imgsz=640)

Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/custom_dataset_config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning /content/drive/MyDrive/custom_images2.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      2.35G     0.8678      2.137      1.098          4        640: 100%|██████████| 7/7 [00:01<00:00,  4.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.97it/s]

                   all         50         52      0.709      0.655       0.66      0.458



20 epochs completed in 0.023 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.83it/s]


                   all         50         52      0.714      0.624      0.668      0.462
Speed: 0.3ms preprocess, 2.5ms inference, 0.0ms loss, 6.1ms postprocess per image
Results saved to runs/detect/train3


In [ ]:
results = model.model_medium_pt.train(data = '/content/drive/MyDrive/custom_dataset_config.yaml', epochs=20, imgsz=640)

Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/custom_dataset_config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tru

train: Scanning /content/drive/MyDrive/custom_images2.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/drive/MyDrive/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      7.38G     0.9893      1.514      1.296          4        640: 100%|██████████| 7/7 [00:03<00:00,  2.10it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.66it/s]

                   all         50         52      0.186      0.538      0.195     0.0762



20 epochs completed in 0.036 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 52.0MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.98it/s]


                   all         50         52      0.642      0.898      0.768      0.685
Speed: 0.2ms preprocess, 10.2ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/detect/train4


mAP50 Parameter Values obtained after 20 Epochs for all models are as follows-

**Nanov8 from Scratch (Complete Dataset)** - 0.217

**Nanov8 Pretrained Model (Complete Dataset)** - 0.771

**Mediumv8 Pretrained Model (Complete Dataset)** - 0.67

**Nanov8 from Scratch (Custom Dataset)** - 0.0288

**Nanov8 Pretrained Model (Custom Dataset)** - 0.668

**Mediumv8 Pretrained Model (Custom Dataset)** - 0.768


In general, it is beneficial to train your YOLOv8 object detection model on a larger dataset. This is the reason why:

**More data, better learning**: Your model is exposed to a greater range of objects, backdrops, and lighting situations when it is part of a larger dataset. It performs better on unseen photos as a result of learning more robust characteristics and becoming less prone to overfit to the training set.

**Accuracy sharpening**: More samples allow the model to collect more minute details in object features, resulting in increased precision in detection and classification. Additionally, it is more likely to come across uncommon objects, which better prepares it for real-world situations.

However, there are several drawbacks:

**Diminishing returns**: Increasing data is beneficial for a while, but after that, it becomes ineffective. Accuracy may not be appreciably improved by more data.
Data quality is important. If a large dataset has inadequate labelling, it is of no utility. Well-annotated, clean data is essential.

Here the Bigger Model, in 2 out of the 3 models perform better, so in general, the bigger model performs better because as the number of training samples increases, the features learnt by the model to detect the objects in the Image also increases, which leads to better generalisation over the Training Set, which leads to higher precision and higher performance metrics in general.

In [10]:
model.model_nano_scratch.val(data='/content/drive/MyDrive/config.yaml', imgsz=640, conf=0.25, iou=0.5)

Ultralytics YOLOv8.1.45 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


val: Scanning /content/drive/MyDrive/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.02it/s]


                   all         50         52       0.75     0.0577        0.4      0.198
Speed: 0.4ms preprocess, 33.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7d7ef8330400>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [8]:
from IPython.display import Image 
Image(url='./validation_visualisation/21086afedd53661f.png', width=300, height=300) 

In [9]:
Image(url='./validation_visualisation/02b336d12bc2674.png', width=300, height=300) 

In [10]:
Image(url='./validation_visualisation/11520a5f881bfead.png', width=300, height=300) 

In [11]:
Image(url='./validation_visualisation/14d3556c1249439d.png', width=300, height=300)

The Visualisations of the Above images on the Validation Set likely signifies that the Model has more or less started learning the Features of the Duck and has started to draw the Bounding Boxes encapsulating the Ducks. However, the confidence Scores are mediocre and not very high. This means that the Model is not trained enough to completely identify objects and draw boxes around them. Since the Model was trained only for 20 epochs and that too for only 400 images in the Dataset and the dataset used was Nanov8 Model from Scratch, Keeping all these things in mind, we can say that model performs decently well and is able to capture the underlying features of the images, and with more training would be able to draw boxes around the object in a better fashion.

### 2.5 Impact of Augmentations

Default run of YOLOV8 models (YOLOv8n from Scratch, YOLOV8n from Pretrained Model, YOLOV8m from Pretrained Model) use Augmentations. They use Albumentations Library to perform Data Augmentation. They use the following Augmentations -

Blur(p=0.01, blur_limit=(3, 7)): Applies a blur effect to the image using blur kernel whose size is chosen randomly between 3 and 7 pixels (inclusive). Blurring introduces variations in image clarity, making the model more robust to slight defocus in real-world images.

MedianBlur(p=0.01, blur_limit=(3, 7)): Median blur is a noise reduction technique that can help the model differentiate objects from noise in the image.

ToGray(p=0.01): While YOLOv8 works with color images, training with grayscale versions can improve the model's ability to detect objects based on shapes and edges, rather than relying solely on color information. So, it's task is to convert colored images to grayscale images.

CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8)): Contrast Limited Adaptive Histogram Equalization (CLAHE) improves local contrast in images, which can be helpful for highlighting objects in unevenly lit scenes. The parameters here define the clip limit for contrast enhancement (1 to 4) and the grid size used for local processing (8x8 tiles).

All of these Augmentation Techniques are applied with a probability of 1% because p is set as 0.01.
Overall, these augmentations artificially create variations in your training data. This helps the YOLOv8 model learn features that are generalizable to unseen images and reduces the risk of overfitting to the specific training dataset.

In [ ]:
!pip3 uninstall albumentations

Found existing installation: albumentations 1.3.1
Uninstalling albumentations-1.3.1:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/albumentations-1.3.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/albumentations/*
Proceed (Y/n)? y
  Successfully uninstalled albumentations-1.3.1


In [ ]:
results = model.model_nano_pt.train(data = '/content/drive/MyDrive/custom_dataset_config.yaml', epochs=20, imgsz=640)

Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/custom_dataset_config.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

100%|██████████| 755k/755k [00:00<00:00, 16.1MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/drive/MyDrive/custom_images2.cache... 100 images, 0 backgrounds, 0 corrupt: 100%|██████████| 100/100 [00:00<?, ?it/s]
val: Scanning /content/drive/MyDrive/labels/val.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      2.29G     0.8078      2.056      1.084          4        640: 100%|██████████| 7/7 [00:01<00:00,  4.78it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.52it/s]

                   all         50         52      0.643      0.577       0.64      0.449



20 epochs completed in 0.025 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.44 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.93it/s]


                   all         50         52      0.642      0.577      0.639      0.451
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train


In [15]:
!cp -r runs/detect/train4 drive/MyDrive/

mAP50 Parameter Values for Nanov8 Pretrained YOLO Model trained on 20 epochs with Custom Dataset of 100 Images are as follows - 

**Nanov8 Pretrained Model (Custom Dataset) with Augmentations** - 0.668

**Nanov8 Pretrained Model (Custom Dataset) without Augmentations** - 0.639

We observe that the change in performance is not that significant. It is because all the default augmentations, though they are applied are run with a probability of only 1%. This means that on an average, only on 1 image out of 100 samples will the augmentation be applied. This helps the model to generalise better but is not significantly impacting the model since new features learnt using augmentations are very limited.  

Due to the liitation of ultralytics library, we cannot change the default augmenations and just remove them by uninstalling albumentations library.
Out of the augmentations 1. Blur(p=0.01, blur_limit=(3, 7)) 2. MedianBlur(p=0.01, blur_limit=(3, 7)) 3.ToGray(p=0.01) 4.CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8)) for YOLOv8 object detection, the most important ones are likely: 

1. **ToGray**: Prevents model to focus on colors which is not important wince ducks are found in various colors in dataset.

2. **ImageCompression** : May degrade quality of image but will remove unnessecary features.

3. **CLAHE** : YOLOv8 works on the principle of Contrastive differentiation. That is, it is not able to differentiate between lighting in the images solely. CLAHE augmentation improves contrast in the images and thus is helpful to improve the contrast of lighting sources in the images which would allow YOLO to distinguish images having almost the same light surroundings.

We determined this because the Dataset is very small and the impact of the Contrastive augmentations will be more pronounced than the blurring augmentations which would lower the already low power of contrast of YOLO, because the model is very likely to overfit on small datasets and the first 2 augmentations would not really help us in solving the task.